In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
import pyspark.sql.functions as func
import pandas as pd

In [2]:
sc = SparkContext(appName="discount analysis")
spark = SparkSession(sc)

In [5]:
input_df = spark.read.csv("input.txt")

# review - price history

In [46]:
import os
input_path = "../../data/review_price/review-download/output/"
input_list = os.listdir(input_path)
with open("input.txt", 'w', encoding='utf-8') as f:
    for file in input_list:
        f.write(file+'\n')

In [47]:
input_df = spark.read.csv("input.txt")
input_df = input_df.toPandas()
input_list = input_df[input_df.columns[0]].to_list()
print(type(input_list))
input_list[:10]

<class 'list'>


['10180.csv',
 '10220.csv',
 '102600.csv',
 '104900.csv',
 '10500.csv',
 '105000.csv',
 '105600.csv',
 '10680.csv',
 '107100.csv',
 '107200.csv']

In [48]:
df = spark.read.csv(input_path+"240.csv", header=True, inferSchema=True)
df.show(10)

+-------------------+--------+-----+-------------+--------+-------------+--------------+--------------+------------+
|               date|  Owners|Price|PriceChangeTo|discount|total_reviews|total_positive|total_negative|review_score|
+-------------------+--------+-----+-------------+--------+-------------+--------------+--------------+------------+
|2015-04-01 00:00:00|13698000|  0.0|        19.99|    null|        34324|         33346|           978|           9|
|2015-06-11 00:00:00|13830000|19.99|         7.49|   37.47|        36287|         35177|          1110|           9|
|2015-06-22 00:00:00|13893000| 7.49|        19.99|  266.89|        36806|         35677|          1129|           9|
|2015-11-25 00:00:00|14300000|19.99|         7.49|   37.47|        40673|         39321|          1352|           9|
|2015-12-01 00:00:00|14314000| 7.49|        19.99|  266.89|        40938|         39569|          1369|           9|
|2015-12-22 00:00:00|14362000|19.99|         7.49|   37.47|     

In [49]:
df.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- Owners: integer (nullable = true)
 |-- Price: double (nullable = true)
 |-- PriceChangeTo: double (nullable = true)
 |-- discount: double (nullable = true)
 |-- total_reviews: integer (nullable = true)
 |-- total_positive: integer (nullable = true)
 |-- total_negative: integer (nullable = true)
 |-- review_score: integer (nullable = true)



In [50]:
df = df.dropna()
df.show(10)

+-------------------+--------+-----+-------------+--------+-------------+--------------+--------------+------------+
|               date|  Owners|Price|PriceChangeTo|discount|total_reviews|total_positive|total_negative|review_score|
+-------------------+--------+-----+-------------+--------+-------------+--------------+--------------+------------+
|2015-06-11 00:00:00|13830000|19.99|         7.49|   37.47|        36287|         35177|          1110|           9|
|2015-06-22 00:00:00|13893000| 7.49|        19.99|  266.89|        36806|         35677|          1129|           9|
|2015-11-25 00:00:00|14300000|19.99|         7.49|   37.47|        40673|         39321|          1352|           9|
|2015-12-01 00:00:00|14314000| 7.49|        19.99|  266.89|        40938|         39569|          1369|           9|
|2015-12-22 00:00:00|14362000|19.99|         7.49|   37.47|        41547|         40150|          1397|           9|
|2016-01-04 00:00:00|14444000| 7.49|        19.99|  266.89|     

In [51]:
df.columns

['date',
 'Owners',
 'Price',
 'PriceChangeTo',
 'discount',
 'total_reviews',
 'total_positive',
 'total_negative',
 'review_score']

In [52]:
df.select('discount').show(5)

+--------+
|discount|
+--------+
|   37.47|
|  266.89|
|   37.47|
|  266.89|
|   37.47|
+--------+
only showing top 5 rows



In [53]:
df = df.withColumn("discount", func.round((df.Price - df.PriceChangeTo) / df.Price * 100, 0).cast('Int'))
df.show(10)

+-------------------+--------+-----+-------------+--------+-------------+--------------+--------------+------------+
|               date|  Owners|Price|PriceChangeTo|discount|total_reviews|total_positive|total_negative|review_score|
+-------------------+--------+-----+-------------+--------+-------------+--------------+--------------+------------+
|2015-06-11 00:00:00|13830000|19.99|         7.49|      63|        36287|         35177|          1110|           9|
|2015-06-22 00:00:00|13893000| 7.49|        19.99|    -167|        36806|         35677|          1129|           9|
|2015-11-25 00:00:00|14300000|19.99|         7.49|      63|        40673|         39321|          1352|           9|
|2015-12-01 00:00:00|14314000| 7.49|        19.99|    -167|        40938|         39569|          1369|           9|
|2015-12-22 00:00:00|14362000|19.99|         7.49|      63|        41547|         40150|          1397|           9|
|2016-01-04 00:00:00|14444000| 7.49|        19.99|    -167|     

In [54]:
df = df.withColumn('app_id', func.lit('240'))
df.show()

+-------------------+--------+-----+-------------+--------+-------------+--------------+--------------+------------+------+
|               date|  Owners|Price|PriceChangeTo|discount|total_reviews|total_positive|total_negative|review_score|app_id|
+-------------------+--------+-----+-------------+--------+-------------+--------------+--------------+------------+------+
|2015-06-11 00:00:00|13830000|19.99|         7.49|      63|        36287|         35177|          1110|           9|   240|
|2015-06-22 00:00:00|13893000| 7.49|        19.99|    -167|        36806|         35677|          1129|           9|   240|
|2015-11-25 00:00:00|14300000|19.99|         7.49|      63|        40673|         39321|          1352|           9|   240|
|2015-12-01 00:00:00|14314000| 7.49|        19.99|    -167|        40938|         39569|          1369|           9|   240|
|2015-12-22 00:00:00|14362000|19.99|         7.49|      63|        41547|         40150|          1397|           9|   240|
|2016-01

In [55]:
# df.write.options(header='True', delimiter=',').csv("./ouput")

In [ ]:
id1 = input_list[0]
df1 = spark.read.csv(input_path+id1, header=True, inferSchema=True)
df1 = df1.withColumn('app_id', func.lit(id1))
df1.show()

In [ ]:
id2 = input_list[1]
df2 = spark.read.csv(input_path+id2, header=True, inferSchema=True)
df2 = df2.withColumn('app_id', func.lit(id2))
df2.show()

In [ ]:
union_df = df1.union(df2)
union_df.show(100)